In [2]:
import quimb as qu
import numpy as np

/home/niall/.local/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:34: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [3]:
# We're using the quimb library here but you could equally use just numpy
# Quimb provides us with some wrappers around numpy specifically for quantum information

bell_state_vector = (1/np.sqrt(2))*qu.qarray([[0],[1],[-1],[0]])
bell_density_mat  = bell_state_vector@bell_state_vector.H

print("Bell State Vector:")
print(bell_state_vector)
print("Bell Density Matrix:")
print(bell_density_mat)

Bell State Vector:
[[ 0.      ]
 [ 0.707107]
 [-0.707107]
 [ 0.      ]]
Bell Density Matrix:
[[ 0.   0.   0.   0. ]
 [ 0.   0.5 -0.5  0. ]
 [ 0.  -0.5  0.5  0. ]
 [ 0.   0.   0.   0. ]]


In [4]:
# We should always check that our density matrices are valid
# Let's define a function to do this for us based on the conditions outlined in the notes

def is_valid_rho(rho):
    # Check if Hermitian
    is_hermitian   = np.array_equal(rho.H, rho)

    # Calculate the trace and check if equal to 1
    trace_rho      = np.isclose(qu.trace(rho), 1)

    # Calculate eigenvalues and check if non-negative
    eigvals_rho    = np.linalg.eigvals(rho)
    eigvals_nonneg = all(eigval >= 0 for eigval in eigvals_rho)

    # Condition for valid density matrix
    return (is_hermitian and trace_rho and eigvals_nonneg)

# Let's test this on our bell state example
print(f"Valid Density Matrix (Bell): {is_valid_rho(bell_density_mat)}")

# Let's define some more density matrices and test these
rho_test_1 = qu.qarray([[3/4, 1/4],[1/4, 1/4]])
rho_test_2 = qu.qarray([[1.2, 0],[0, -0.2]])

print(f"Valid Density Matrix (test_1): {is_valid_rho(rho_test_1)}")
print(f"Valid Density Matrix (test_2): {is_valid_rho(rho_test_2)}")

Valid Density Matrix (Bell): True
Valid Density Matrix (test_1): True
Valid Density Matrix (test_2): False


In [5]:
# Once we have a valid density matrix we can then check if it represents a pure or mixed state
# Let's define another function to do this for us that returns the purity of our state
def purity(rho):
    if is_valid_rho(rho):
        # Compute rho**2
        rho_squared  = rho@rho
        state_purity = qu.trace(rho_squared)
    else:
        state_purity = -1 #Choose an invalid value for purity if invalid density matrix
    return state_purity

# Recall if the purity is 1 then we have a pure state
# If the purity is less than 1 and greater than 0.5 then we have a mixed state
# The purity is non-negative so if we get -1 then we provided an invalid density matrix

#Let's look at the purity of the bell density matrix
bell_purity = purity(bell_density_mat)
print("Bell State")
if np.isclose(bell_purity, 1):
    print(f"Pure State: p = {bell_purity}")
else:
    print(f"Mixed State: p = {bell_purity}")

#Let's look at the purity of the rho_test_1 state
test_1_purity = purity(rho_test_1)
print("rho_test_1")
if np.isclose(test_1_purity, 1):
    print(f"Pure State: p = {test_1_purity}")
else:
    print(f"Mixed State: p = {test_1_purity}")

Bell State
Pure State: p = 0.9999999999999996
rho_test_1
Mixed State: p = 0.75
